In [2]:
import pandas as pd
import nbashots as nba
import json
import requests
import csv
from __future__ import division

In [7]:
player_shots = nba.Shots()
player_shots.update_params({'DateFrom':'01/01/16'})
player_shots.update_params({'Dateto':'01/31/16'})
df_players = player_shots.get_shots()

In [4]:
cousin_id = nba.get_player_id("Cousins, DeMarcus")

jan_shots = nba.Shots(player_id=cousin_id)
jan_shots.update_params({'DateFrom':'01/01/16'})
jan_shots.update_params({'Dateto':'01/31/16'})
cousin_jan_shot = jan_shots.get_shots()

In [8]:
players = []
players = pd.unique(df_players.PLAYER_ID.ravel())
len(players)

423

In [10]:
cnt = 0
players_log = nba.PlayerLog(players[cnt])

#grabs PlayerLog for current player converts the GAME DATE into a real date variable then limits it to only set dates
player_df = players_log.get_game_logs()
player_df['GAME_DATE'] = player_df['GAME_DATE'].astype('datetime64[ns]')
newdf = player_df[(player_df['GAME_DATE']>'2015-12-31') & (player_df['GAME_DATE']<'2016-02-01')]

#pulls individual player stats
rebounds =newdf['REB'].sum()
total_assist = newdf['AST'].sum()
total_BLK = newdf['BLK'].sum()
total_pts = newdf['PTS'].sum()
free_throws_made = newdf['FTM'].sum()
fg_attempted = newdf['FGA'].sum()
avg_plus_minus = newdf['PLUS_MINUS'].mean()
avg_free_pct = newdf['FT_PCT'].mean()
avg_field_pct = newdf['FG_PCT'].mean()
total_minutes = newdf['MIN'].sum()
avg_minutes = newdf['MIN'].mean()

#conversions for adjusted stats to miuntes played
rebound_by_min = rebounds / total_minutes * 36
assist_by_min = total_assist / total_minutes * 36
block_by_min = total_BLK / total_minutes * 36
pts_by_min = total_pts / total_minutes * 36
ftm_by_min = free_throws_made / total_minutes * 36
fg_atmpt_by_min = fg_attempted / total_minutes * 36

# creates a data frame with our selected metrics to view anll pretty explanitory by name
all_players_jan = pd.DataFrame({ 'Player_ID' : players[cnt],
                                 'PTS' : total_pts,
                                 'AST' : total_assist,
                                 'BLK' : total_BLK,
                                 'REB' : rebounds,
                                 'MIN' : total_minutes,
                                 'FTM' : free_throws_made,
                                 'FGA' : fg_attempted,
                                 'FT_PCT' : avg_free_pct,
                                 'FG_PCT' : avg_field_pct,
                                 '+-AVG' : avg_plus_minus,
                                 'AVMIN' : avg_minutes,
                                 'FT/36' : ftm_by_min,
                                 'FG/36' : fg_atmpt_by_min,
                                 'PTS/36' : pts_by_min,
                                 'AST/36' : assist_by_min,
                                 'BLK/36' : block_by_min,
                                 'REB/36' : rebound_by_min }, index=[cnt])
cnt = cnt + 1

#grabs the rest of the players by iterating through the loop and appending it to the all_players_jan DF
while cnt < len(players):
    players_log.update_params({'PlayerID': players[cnt]})
    player_df = players_log.get_game_logs()
    player_df['GAME_DATE'] = player_df['GAME_DATE'].astype('datetime64[ns]')
    newdf = player_df[(player_df['GAME_DATE']>'2015-12-31') & (player_df['GAME_DATE']<'2016-02-01')]
    
    rebounds =newdf['REB'].sum()
    total_assist = newdf['AST'].sum()
    total_BLK = newdf['BLK'].sum()
    total_pts = newdf['PTS'].sum()
    free_throws_made = newdf['FTM'].sum()
    fg_attempted = newdf['FGA'].sum()
    avg_plus_minus = newdf['PLUS_MINUS'].mean()
    total_minutes = newdf['MIN'].sum()
    avg_minutes = newdf['MIN'].mean()
    avg_free_pct = newdf['FT_PCT'].mean()
    avg_field_pct = newdf['FG_PCT'].mean()
    
    if total_minutes > 0:
        rebound_by_min = rebounds / total_minutes * 36
        assist_by_min = total_assist / total_minutes * 36
        block_by_min = total_BLK / total_minutes * 36
        pts_by_min = total_pts / total_minutes * 36
        ftm_by_min = free_throws_made / total_minutes * 36
        fg_atmpt_by_min = fg_attempted / total_minutes * 36
    
        my_data_jan = pd.DataFrame({ 'Player_ID' : players[cnt],
                                     'PTS' : total_pts,
                                     'AST' : total_assist,
                                     'BLK' : total_BLK,
                                     'REB' : rebounds,
                                     'MIN' : total_minutes,
                                     'FTM' : free_throws_made,
                                     'FGA' : fg_attempted,
                                     'FT_PCT' : avg_free_pct,
                                     'FG_PCT' : avg_field_pct,
                                     '+-AVG' : avg_plus_minus,
                                     'AVMIN' : avg_minutes,
                                     'FT/36' : ftm_by_min,
                                     'FG/36' : fg_atmpt_by_min,
                                     'PTS/36' : pts_by_min,
                                     'AST/36' : assist_by_min,
                                     'BLK/36' : block_by_min,
                                     'REB/36' : rebound_by_min }, index=[cnt])
        #not sure what this does exactly confused why it refernces both DF
        frame = [all_players_jan, my_data_jan]
        #adds the indivdual player into the DF with all our players
        all_players_jan = pd.concat(frame)    
    cnt = cnt + 1
all_players_jan.head()

,+-AVG,AST,AST/36,AVMIN,BLK,BLK/36,FG/36,FGA,FG_PCT,FT/36,FTM,FT_PCT,MIN,PTS,PTS/36,Player_ID,REB,REB/36
0,-6.214286,42,3.315789,32.571429,21,1.657895,17.210526,218,0.432929,1.973684,25,0.565500,456,213,16.815789,202696,128,10.105263
1,-1.500000,19,1.772021,32.166667,18,1.678756,10.352332,111,0.604083,1.958549,21,0.510917,386,151,14.082902,101162,129,12.031088
2,0.333333,41,3.075000,32.000000,6,0.450000,11.100000,148,0.423000,1.650000,22,0.436667,480,173,12.975000,202066,46,3.450000
3,-6.785714,34,2.523711,34.642857,10,0.742268,10.837113,146,0.442143,2.078351,28,0.622643,485,168,12.470103,202699,90,6.680412
4,-11.300000,48,6.400000,27.000000,2,0.266667,12.266667,92,0.410800,1.466667,11,0.405600,270,98,13.066667,203901,30,4.000000


In [11]:
#used to get all the player names and team names to aid in human readability, matches it on Player_ID with all player data calls PERSON_ID
players_df = nba.get_all_player_ids("all_data")
players_df = players_df[players_df["PERSON_ID"].isin(players)]
players_df = players_df.drop(['ROSTERSTATUS','FROM_YEAR','TO_YEAR','PLAYERCODE','TEAM_ID','TEAM_CITY','TEAM_ABBREVIATION','TEAM_CODE','GAMES_PLAYED_FLAG'],axis=1)
players_df = players_df.rename(columns = {'PERSON_ID' : 'Player_ID'})

In [12]:
#adds player name and team 
complete_df = pd.merge(players_df, all_players_jan, on='Player_ID', how='left')
#removes any player who didn't play at least 10 minutes in the month
complete_df = complete_df[complete_df["MIN"] > 10]

In [13]:
#manually entered into a csv file has " " delimter because of , in name must use skipinitialspace
cnt = 0
center_names = []
with open('NBA Centers Player Roster.csv', 'rb') as csvfile:
    nbareader = csv.reader(csvfile, skipinitialspace=True)
    for row in nbareader:
            center_names.append(row[0])

In [14]:
#can compare complete_df for all players or centers_complete_df for just centers
centers_complete_df = complete_df[complete_df["DISPLAY_LAST_COMMA_FIRST"].isin(center_names)]

In [35]:
pd.set_option('display.max_rows', 100)
complete_df.sort_values('PTS', ascending=False)

,Player_ID,DISPLAY_LAST_COMMA_FIRST,TEAM_NAME,+-AVG,AST,AST/36,AVMIN,BLK,BLK/36,FG/36,FGA,FG_PCT,FT/36,FTM,FT_PCT,MIN,PTS,PTS/36,REB,REB/36
86,202326,"Cousins, DeMarcus",Kings,5.733333,47,3.087591,36.533333,19,1.248175,22.598540,344,0.479333,8.145985,124,0.738133,548,473,31.072993,193,12.678832
94,201939,"Curry, Stephen",Warriors,16.000000,106,7.497053,31.812500,6,0.424361,21.005894,297,0.483125,4.243615,60,0.720187,509,435,30.766208,69,4.880157
156,201935,"Harden, James",Rockets,-2.750000,116,7.287958,35.812500,8,0.502618,17.654450,281,0.457688,8.167539,130,0.883063,573,429,26.952880,108,6.785340
112,201142,"Durant, Kevin",Thunder,9.066667,64,4.243094,36.200000,14,0.928177,19.359116,292,0.508933,6.828729,103,0.911000,543,427,28.309392,139,9.215470
63,202710,"Butler, Jimmy",Bulls,1.125000,87,5.100977,38.375000,5,0.293160,16.944625,289,0.445375,6.742671,115,0.817187,614,398,23.335505,93,5.452769
379,202691,"Thompson, Klay",Warriors,15.375000,40,2.716981,33.125000,9,0.611321,19.698113,290,0.483500,3.328302,49,0.773438,530,388,26.354717,65,4.415094
375,202738,"Thomas, Isaiah",Celtics,1.470588,105,6.987061,31.823529,3,0.199630,19.231054,289,0.432765,6.654344,100,0.897765,541,386,25.685767,50,3.327172
397,202689,"Walker, Kemba",Hornets,-0.705882,84,4.885299,36.411765,8,0.465267,18.959612,326,0.395118,5.234249,90,0.869588,619,383,22.274637,74,4.303716
194,2544,"James, LeBron",Cavaliers,9.000000,114,7.328571,35.000000,11,0.707143,16.778571,261,0.565000,4.435714,69,0.732625,560,377,24.235714,105,6.750000
129,201568,"Gallinari, Danilo",Nuggets,1.133333,32,2.181818,35.200000,3,0.204545,15.409091,226,0.439733,8.454545,124,0.855400,528,352,24.000000,75,5.113636


In [34]:
centers_complete_df.sort_values('REB', ascending=False)

,Player_ID,DISPLAY_LAST_COMMA_FIRST,TEAM_NAME,+-AVG,AST,AST/36,AVMIN,BLK,BLK/36,FG/36,FGA,FG_PCT,FT/36,FTM,FT_PCT,MIN,PTS,PTS/36,REB,REB/36
86,202326,"Cousins, DeMarcus",Kings,5.733333,47,3.087591,36.533333,19,1.248175,22.598540,344,0.479333,8.145985,124,0.738133,548,473,31.072993,193,12.678832
108,203083,"Drummond, Andre",Pistons,-1.133333,8,0.628821,30.533333,22,1.729258,13.834061,176,0.524200,2.986900,38,0.264200,458,222,17.449782,189,14.855895
212,201599,"Jordan, DeAndre",Clippers,9.500000,16,1.364929,35.166667,22,1.876777,6.995261,82,0.697667,4.180095,49,0.503083,422,165,14.075829,181,15.440758
302,2585,"Pachulia, Zaza",Mavericks,0.230769,29,2.521739,31.846154,4,0.347826,8.869565,102,0.464308,3.217391,37,0.709000,414,133,11.565217,152,13.217391
273,202328,"Monroe, Greg",Bucks,-0.666667,30,2.416107,29.800000,12,0.966443,16.268456,202,0.519200,5.959732,74,0.783600,447,280,22.550336,151,12.161074
388,202685,"Valanciunas, Jonas",Raptors,2.928571,17,1.602094,27.285714,23,2.167539,11.968586,127,0.509643,3.958115,42,0.653500,382,172,16.209424,145,13.664921
183,2730,"Howard, Dwight",Rockets,-0.333333,14,1.347594,31.166667,14,1.347594,10.780749,112,0.534167,6.160428,64,0.575000,374,196,18.866310,143,13.764706
76,2199,"Chandler, Tyson",Suns,-11.357143,24,2.461538,25.071429,8,0.820513,6.769231,66,0.481000,2.564103,25,0.472643,351,87,8.923077,136,13.948718
240,201572,"Lopez, Brook",Nets,-0.500000,25,1.701323,33.062500,26,1.769376,17.625709,259,0.522062,4.763705,70,0.663687,529,340,23.137996,134,9.119093
144,101162,"Gortat, Marcin",Wizards,-1.500000,19,1.772021,32.166667,18,1.678756,10.352332,111,0.604083,1.958549,21,0.510917,386,151,14.082902,129,12.031088


In [17]:
centers_id = []
centers_id = pd.unique(centers_complete_df.Player_ID.ravel())
centers_shots_stats = df_players[df_players["PLAYER_ID"].isin(centers_id)]

In [18]:
centers_shots_stats.groupby(['SHOT_ZONE_BASIC','SHOT_ZONE_AREA','SHOT_MADE_FLAG']).size()

#this will give you stats for the whole league
#df_players.groupby(['SHOT_ZONE_BASIC','SHOT_ZONE_AREA','SHOT_MADE_FLAG']).size()

SHOT_ZONE_BASIC        SHOT_ZONE_AREA         SHOT_MADE_FLAG
Above the Break 3      Center(C)              0                  129
                                              1                   72
                       Left Side Center(LC)   0                  101
                                              1                   44
                       Right Side Center(RC)  0                   82
                                              1                   56
Backcourt              Back Court(BC)         0                    5
                                              1                    1
In The Paint (Non-RA)  Center(C)              0                  873
                                              1                  639
                       Left Side(L)           0                   63
                                              1                   35
                       Right Side(R)          0                   34
                                          

In [19]:
cousin_jan_shot.groupby(['SHOT_ZONE_BASIC','SHOT_ZONE_AREA','SHOT_MADE_FLAG']).size()

SHOT_ZONE_BASIC        SHOT_ZONE_AREA         SHOT_MADE_FLAG
Above the Break 3      Center(C)              0                  19
                                              1                  10
                       Left Side Center(LC)   0                  11
                                              1                  11
                       Right Side Center(RC)  0                   4
                                              1                   4
In The Paint (Non-RA)  Center(C)              0                  55
                                              1                  33
                       Left Side(L)           0                   4
                                              1                   2
Left Corner 3          Left Side(L)           0                   1
                                              1                   1
Mid-Range              Center(C)              0                  15
                                              1        

In [20]:
#Centers league wide zone shot percentages avg
#Above the break
#    Center - 35.82%  Left Center - 30.34%  Right Center - 40.58%   
#    Left Corner - 27.27%  Right Corner - 32.69%
#Mid Range
#    Center - 42.89%  Left Side Center - 41.22%  Right Side Center - 39.81%  
#    Left Side - 42%  Right Side - 44.64%
#In The Paint
#    Center - 42.46%  Left Side - 35.71%  Right Side- 46.88%
#Rescrited Area
#    Center - 62.4%

#Cousin;s zone shot percentages
#Above the break
#    Center - 34.48%  Left Center - 50%  Right Center - 50%   
#    Left Corner - 50%  Right Corner - N/A
#Mid Range
#    Center - 40%  Left Side Center - 38.1%  Right Side Center - 37.5%
#    Left Side - 36.67%  Right Side - 28.57%
#In The Paint
#    Center - 37.5%  Left Side - 33.33%  Right Side- N/A
#Rescrited Area
#    Center - 58.08%